# Cohere Multi-Step tool use with the Bedrock Converse API


#### Introduction
In this notebook, we demonstrate how to use the Bedrock Converse API with the `list_calendar_events` and `create_calendar_event` tools to book appointments. Booking the correct appointment requires the model to first check for an available slot by listing existing events, reasoning about the correct slot to book the new appointment and then finally invoking the right tool to create the calendar event. All code was orinally provided by Cohere and this notebook supplements using Amazon Bedrock's Converse API.

#### Amazon Bedrock
Amazon Bedrock is a fully managed service that provides access to a wide range of powerful foundational models(FMs) through an API.

In [ ]:
%pip install boto3 IPython

#### Initialize the Bedrock client and required imports

In [ ]:
# Instantiate the Bedrock client
import boto3
import json
bedrock_rt = boto3.client(service_name="bedrock-runtime")

##### Below sets default model to command R+ model with option to change to Command R

In [ ]:
#define command R and command R+ models
DEFAULT_MODEL= "cohere.command-r-plus-v1:0"
COMMAND_R_PLUS = "cohere.command-r-plus-v1:0"
COMMAND_R = "cohere.command-r-v1:0"
model_id = DEFAULT_MODEL

#### Define the tools
##### First let's configure the functions for list calendar events and create calendar events

In [ ]:
# Define the tools

def list_calendar_events(date: str):
    events =  '[{"start": "14:00", "end": "15:00"}, {"start": "15:00", "end": "16:00"}, {"start": "17:00", "end": "18:00"}]'
    print(f"Listing events: {events}")
    return events

def create_calendar_event(date: str, time: str, duration: int):
  print(f"Creating a {duration} hour long event at {time} on {date}")
  return True
  

#### Define the tools
##### Next, we define the json to define the actual tools to then send into Bedrock. This is the standard structure to use for Tool Use with Cohere modles and the Bedrock Converse API.

In [ ]:

list_calendar_events_tool = {
        'name': 'list_calendar_events',
        'description': 'returns a list of calendar events for the specified date, including the start time and end time for each event',
        'inputSchema': {
            'json': {
                'type': 'object',
                'properties': {
                    'date': {
                        'type': 'string',
                        'description': 'the date to list events for, formatted as mm/dd/yy'
                    }
                },
                'required': [
                    'date'
                ]
            }
        }
    }


create_calendar_event_tool = {
        'name': 'create_calendar_event_tool',
        'description': 'creates a calendar event of the specified duration at the specified time and date',
        'inputSchema': {
            'json': {
                'type': 'object',
                'properties': {
                    'date': {
                        'type': 'string',
                        'description': 'the date to list events for, formatted as mm/dd/yy'
                    },
                    'time': {
                        'type': 'string',
                        'description': 'the time of the event, formatted using 24h military time formatting'
                    },
                    'duration': {
                        'type': 'float',
                        'description': 'the number of hours the event lasts for'
                    }
                    
                },
            
                'required': [
                    'date', 'time', 'duration'
                ]
            }
              
            }
        }

##### Take note of how the tools are then defined in the tool_config variable. We will create the toolConfig parameter by calling the variable "tool_config" and passing it into the Converse API parameter to set toolConfig

In [ ]:
tool_config = {'tools': []}

tool_config['tools'].append({
    'toolSpec': list_calendar_events_tool
})


tool_config['tools'].append({
    'toolSpec': create_calendar_event_tool
               
})

##### Define the tool function to identify which tool to call

In [ ]:
def invoke_tool(tool_call):
  # Iterate over the tool calls generated by the model
    if tool_call['name'] == 'list_calendar_events':
        date = tool_call['input']['date']
        return [{
             "events": list_calendar_events(date)
          }]
    elif tool_call['name'] == 'create_calendar_event_tool':
        date = tool_call['input']['date']
        time = tool_call['input']['time']
        duration = tool_call['input']['duration']
        return [{
             create_calendar_event(date, time, duration)
           }]
    else:
        raise f"Unnown tool name '{tool_call['name']}'"

#### Then, call the Converse API and return the output response. Append each response to the list of messages to keep the standardized format for Bedrock APIs

In [ ]:
# Check what tools the model wants to use and how to use them
additional_model_fields ={"preamble": "Today is Thursday, may 23, 2024"}
input_text = "book an hour long appointment for the first available free slot after 3pm"

#create an initial message to pass the input text into. This is standard format for messages parameter in Converse API
messages = []
initial_message = {
    "role": "user",
    "content": [
        { "text": input_text } 
    ],
}
messages.append(initial_message)

#call the converse API by passing in the additional parameters and the tool_config set earlier
res = bedrock_rt.converse(
    modelId=model_id,
    messages=messages,
    additionalModelRequestFields=additional_model_fields,
    toolConfig=tool_config)

response_message = res['output']['message']
#we want to pass this response_message back in
response_content = response_message['content']
messages.append(response_message)

In [ ]:
#Now will be instantiating which tools to access and passing the response back into the API.
follow_up_message = []
print(response_content[0]['text'])
for content in response_message['content']:
    if 'toolUse' in content:
        tool_use_block = content['toolUse']
        tool_use_name = tool_use_block['name']
        
        #call the function to invoke which tool to use
        tool_result_value = invoke_tool(tool_call)

        follow_up_message.append({
                "toolResult": {
                    "toolUseId": tool_use_block['toolUseId'],
                    "content": [
                        {
                            "json": {
                                "result": tool_result_value
                            }
                        }
                    ]
                }
            })

if len(follow_up_message) > 0:
    
    follow_up_message = {
        "role": "user",
        "content": follow_up_message,
    }
    
    messages.append(follow_up_message)

    response = bedrock_rt.converse(
        modelId= model_id,
        messages=messages,
        toolConfig= tool_config
    )
    
    response_message = response['output']['message']
    messages.append(response_message)

#### Conclusion
This notebook follows the official Cohere Cookbooks and converts the Tool_Use notebook to use the Bedrock Converse API. In this notebook we explored multi-tool use as well as bring in conversation history from an initial output into a second turn in the conversation